<a href="https://colab.research.google.com/github/Adrok24/Proyecto-Integrador/blob/main/Preprocessing_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
!unrar x '/content/drive/My Drive/Colab Notebooks/desafio final/txt_books_spa.rar'


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/drive/My Drive/Colab Notebooks/desafio final/txt_books_spa.rar

Extracting  La ciudad invisible - Arnaldo Visconti.txt                     2%  OK 
Extracting  La esposa complaciente - Barbara Cartland.txt                  5%  OK 
Extracting  La gran moral - Aristoteles.txt                                8%  OK 
Extracting  La lampara roja - Arthur Conan Doyle.txt                      13%  OK 
Extracting  La liga de los pelirrojos - Arthur Conan Doyle.txt            14%  OK 
Extracting  Las afueras de Dios - Antonio Gala.txt                        23%  OK 
Extracting  Los muertos no aceptan preguntas - Antonia Romero.txt         29%  OK 
Extracting  Nadie debe saberlo - Antonio Carrere Garriga.txt              34%  OK 
Extracting  Telepata - Arthur Sellings.txt                                39%  OK 
Extracting  Te

In [54]:
from os import listdir
from os.path import isfile, join

mypath = '/content/'

list_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [55]:
list_files

['La ciudad invisible - Arnaldo Visconti.txt',
 'La esposa complaciente - Barbara Cartland.txt',
 'Las afueras de Dios - Antonio Gala.txt',
 'Los muertos no aceptan preguntas - Antonia Romero.txt',
 'La ciudad del mar - Anton Gill.txt',
 'Tristan e Iseo - Anonimo.txt',
 '50 cosas que hay que saber sobre filosofia - Ben Dupre.txt',
 'Argo - Antonio J. Mendez.txt',
 'El camino a Itaca - Ben Pastor.txt',
 'La lampara roja - Arthur Conan Doyle.txt',
 '_Vale la pena vivir_ - Antonio Rittscher.txt',
 'El pacto - Antonio Martinez Asensio.txt',
 'La gran moral - Aristoteles.txt',
 'Dicen que era yo - Antonio Orozco Guerrero.txt',
 'En el punto de mira - Arthur Miller.txt',
 'La liga de los pelirrojos - Arthur Conan Doyle.txt',
 'Textos hermeticos - Anonimo.txt',
 'Telepata - Arthur Sellings.txt',
 'Nadie debe saberlo - Antonio Carrere Garriga.txt',
 'Un mago en la corte - Anonimo.txt']

In [11]:
!pip install nltk

In [13]:
 from nltk.tokenize import sent_tokenize, word_tokenize

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [95]:
def get_lines(line, book_txt, dict_book_final, count_lines):
  if (len(line) > 180):
    split_line = line.split(',')
    for s_line in split_line:
      if ((len(s_line) < 80) and (len(s_line) > 30)):
        dict_book_final[count_lines] = {"line": s_line, "len": len(s_line), "book": book_txt}
        count_lines = count_lines + 1
  elif (len(line) > 30):  
    dict_book_final[count_lines] = {"line": line, "len": len(line), "book": book_txt}
    count_lines = count_lines + 1
  return dict_book_final, count_lines

In [99]:
dict_book_final = {}
count_lines = 0
for book_txt in list_files:
  corpus = open(book_txt, "r")
  cadena = corpus.read()
  list_cadena = sent_tokenize(cadena, language='spanish')
  print(len(list_cadena),book_txt)

  for line in list_cadena:
    book_txt = book_txt.replace('.txt', '')
    if (line.find('\n') == -1):
      dict_book_final, count_lines = get_lines(line, book_txt, dict_book_final, count_lines)
    else:
      split_line = line.split('\n')
      for s_line in split_line:
        dict_book_final, count_lines = get_lines(s_line, book_txt, dict_book_final, count_lines)

1569 La ciudad invisible - Arnaldo Visconti.txt
3893 La esposa complaciente - Barbara Cartland.txt
6120 Las afueras de Dios - Antonio Gala.txt
8136 Los muertos no aceptan preguntas - Antonia Romero.txt
6242 La ciudad del mar - Anton Gill.txt
3803 Tristan e Iseo - Anonimo.txt
2450 50 cosas que hay que saber sobre filosofia - Ben Dupre.txt
4439 Argo - Antonio J. Mendez.txt
8526 El camino a Itaca - Ben Pastor.txt
4570 La lampara roja - Arthur Conan Doyle.txt
5359 _Vale la pena vivir_ - Antonio Rittscher.txt
3770 El pacto - Antonio Martinez Asensio.txt
1388 La gran moral - Aristoteles.txt
1667 Dicen que era yo - Antonio Orozco Guerrero.txt
6031 En el punto de mira - Arthur Miller.txt
593 La liga de los pelirrojos - Arthur Conan Doyle.txt
7219 Textos hermeticos - Anonimo.txt
4961 Telepata - Arthur Sellings.txt
4838 Nadie debe saberlo - Antonio Carrere Garriga.txt
4206 Un mago en la corte - Anonimo.txt


In [100]:
df_book_lines = pd.DataFrame.from_dict(dict_book_final, "index")
display(df_book_lines)

,line,len,book
0,Colección El Pirata Negro n.° 27,32,La ciudad invisible - Arnaldo Visconti
1,El desierto parecía no tener fin.,33,La ciudad invisible - Arnaldo Visconti
2,"Largos días de camino, mecidos los nómadas por...",123,La ciudad invisible - Arnaldo Visconti
3,Noches frías en campamentos sin lumbre,38,La ciudad invisible - Arnaldo Visconti
4,en que el sueño llegaba calladamente,37,La ciudad invisible - Arnaldo Visconti
...,...,...,...
79806,[1] Los magos o malabaristas eran llamados ent...,104,Un mago en la corte - Anonimo
79807,[3] La epacta o añalejo es el calendario para ...,115,Un mago en la corte - Anonimo
79808,[5] La cencerrada consiste en rondar a los nov...,96,Un mago en la corte - Anonimo
79809,Se canta y baila a la puerta de su casa la vís...,91,Un mago en la corte - Anonimo


In [101]:
df_book_lines.to_csv('/content/drive/My Drive/Colab Notebooks/desafio final/train_corpus.csv')